1. Explore Label Data

In [ ]:
#imports

import os
import matplotlib as mpl
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
import cv2

In [ ]:

# counting summary aspects of data

video_data = os.listdir('..\\data_scraping\\Web_Scrape_Labeled_2\\')
video_data = video_data + os.listdir('..\\data_supplement\\Self_Record_Labeled\\')
# print(video_data)

o_counter = 0
f_counter = 0
b_counter = 0
s_counter = 0
l_counter = 0

left_counter = 0
NUM_FILES = 76

for filename in video_data:
    o_counter += int(filename[4])
    f_counter += int(filename[7])
    b_counter += int(filename[10])
    s_counter += int(filename[13])
    l_counter += int(filename[16])
    if (filename[18] == 'L'): 
        left_counter += 1
    
    # making sure labels are OK after averages
    if (int(filename[7]) > 0 and int(filename[10]) > 0):
        print('Review ' + str(filename))

left_counter = float(left_counter) / (float(NUM_FILES))
left_counter *= 100


In [ ]:
# displaying summative data information

print('L entrances: ' + str(round(left_counter, 3)) + '%')
print('R entrances: ' + str(round(100 - left_counter, 3)) + '%\n')

print('Sums of all ratings: ')
print('overstride: ' + str(o_counter))
print('forward lean: ' + str(f_counter))
print('backward lean: ' + str(b_counter))
print('sweeping: ' + str(s_counter))
print('low arms: ' + str(l_counter))

errors = ['overstride', 'forward lean', 'backward lean', 'sweeping', 'low arms']
rating_sums = [o_counter, f_counter, b_counter, s_counter, l_counter]

fig = plt.figure(figsize=(10, 5))

plt.bar(errors, rating_sums, width = 0.5)
plt.xlabel('Error Type')
plt.ylabel('Sum of Error Ratings')
plt.title('Sums of Error Ratings across 207 Web-Scraped Training Examples')    

for index, value in enumerate(rating_sums):
    plt.text(index - 0.08, value -50, str(value), color='white') # add annotations

plt.show()

In [ ]:
ratings = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

type_count = np.zeros((5, 10), dtype=int)

for filename in video_data:
    type_count[0][int(filename[4])] += 1
    type_count[1][int(filename[7])] += 1
    type_count[2][int(filename[10])] += 1
    type_count[3][int(filename[13])] += 1
    type_count[4][int(filename[16])] += 1

# print(type_count)



fig, ax = plt.subplots()
im = ax.imshow(type_count, cmap='plasma')

ax.set_xticks(np.arange(len(ratings)))
ax.set_yticks(np.arange(len(errors)))
ax.set_xticklabels(ratings)
ax.set_yticklabels(errors)

for i in range(len(errors)):
    for j in range(len(ratings)):
        text = ax.text(j, i, type_count[i, j], ha="center", va="center", color="white")

plt.show()

2. Use MoveNet to Save Time Series

In [ ]:
# visualizing keypoints
def draw_keypoints(frame, keypoints, min_confidence):
    y, x, c = frame.shape
    print ('height: ' + str(y) + ' width: ' + str(x))
    print('KEYPOINTS:')
    print(keypoints)
    shaped = np.squeeze(np.multiply(keypoints, [1, 1, 1]))

    print(shaped[0])

    for kp in shaped:
        ky, kx, conf = kp
        print('KY, KX, CONF: ' + str(int(ky * y)) + ' ' +  str(int(kx * x)) + ' ' + str(conf))
        if conf > min_confidence:
            cv2.circle(frame, (int(kx * x), int(ky * y)), 5, (255, 0, 0), -1)

In [ ]:
# pose estimation

interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_tflite_float16_4.tflite')
interpreter.allocate_tensors()

filename = '..\\data_scraping\\Web_Scrape_Labeled_2\\WS-O3-F1-B0-S4-L1-L-0166.mp4'

cap = cv2.VideoCapture(filename)

while (cap.isOpened()):
    ret, frame = cap.read()

    red = frame[:,:,2].copy()
    blue = frame[:,:,0].copy()
    frame[:,:,2] = blue
    frame[:,:,0] = red

    if ret:
    
        # reshape image
        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256,256)
        input_image = tf.cast(img, dtype=tf.uint8) # change base on lightning/thunder float32/uint8
        
        # set up input / output
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # make prediction
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        points_with_scores = interpreter.get_tensor(output_details[0]['index'])

        print(points_with_scores)

        # display prediction
        square_img = np.array(tf.cast(np.squeeze(img), dtype=tf.uint8))

        square_frame = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), 1280,1280)
        square_frame = np.array(tf.cast(np.squeeze(square_frame), dtype=tf.uint8))


        draw_keypoints(square_frame, points_with_scores, 0.1)  
        
        plt.figure(figsize=(10,10))
        plt.imshow(square_frame) # display only

        cap.release()     
    
    else:
        break

cap.release()

    